Name: Chandrish <br>
Analysis Goal: Training a song embedding based on playlists data and show recommendations of songs. <br>
Algorithm/ Function used: Word2Vec MLLib

In [ ]:
# Libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
from datetime import datetime
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, CountVectorizer
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import os

In [ ]:
# creating spark session
spark = SparkSession.builder.\
                    appName("spotify").\
                    config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1").\
                    config("spark.mongodb.input.uri", "mongodb+srv://**********").\
                    config("spark.mongodb.output.uri", "mongodb+srv://**********").\
                    config("spark.network.timeout", "7200s").\
                    config("spark.executor.heartbeatInterval", "1200s").\
                    getOrCreate()

In [ ]:
# connection string for mongodb
database = '*****'
collection = 'songs_lyrics'
user_name = '*****'
password = '*****'
address = '**********.mongodb.net'
connection_string = f"mongodb+srv://{user_name}:{password}@{address}/{database}.{collection}"
connection_string

Reading playlist data from MongoDB

In [ ]:
# reading the data from Mongodb
df = spark.read.format('mongo').option("uri",connection_string).load()

In [ ]:
# grouping playlist to create array of songs
df_play = df.groupBy('pid').agg(collect_list('track_uri').alias('song_list'))
df_play.cache()

Out[5]: DataFrame[pid: bigint, song_list: array<string>]

In [ ]:
df_play.show(5)

+----+--------------------+
 pid| song_list|
+----+--------------------+
 26|[spotify:track:2J...|
 29|[spotify:track:1O...|
 474|[spotify:track:1R...|
 964|[spotify:track:2b...|
1677|[spotify:track:3l...|
+----+--------------------+
only showing top 5 rows

Word2Vec training, with default parameters for window size and other parameters

In [ ]:
from pyspark.ml.feature import Word2Vec
word2Vec = Word2Vec(vectorSize=32, seed=42, inputCol="song_list").setMinCount(1)
word2Vec.setMaxIter(10)
model = word2Vec.fit(df_play)

Model Execution time: 8.42 hours for 1 million playlists for 10 iterations <br>

In [ ]:
# s3 credentials to connect
ACCESS_KEY = "*****"
SECRET_KEY = "*****"
spark._jsc.hadoopConfiguration().set("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.1")
spark._jsc.hadoopConfiguration().set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
spark._jsc.hadoopConfiguration().set("fs.s3a.access.key", ACCESS_KEY)
spark._jsc.hadoopConfiguration().set("fs.s3a.secret.key", SECRET_KEY)

In [ ]:
# renaming the model output
vectors = model.getVectors().withColumnRenamed('word', 'track_uri')
vectors.cache()
vectors.show(10)

/databricks/spark/python/pyspark/sql/context.py:134: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
 warnings.warn(
+--------------------+--------------------+
 track_uri| vector|
+--------------------+--------------------+
spotify:track:7Bw...|[0.17075945436954...|
spotify:track:3QV...|[-0.1242264881730...|
spotify:track:2wf...|[-0.1510251611471...|
spotify:track:7KX...|[-0.0403408631682...|
spotify:track:23Q...|[-0.5807483792304...|
spotify:track:2gw...|[0.00845383573323...|
spotify:track:3dC...|[-0.1968882828950...|
spotify:track:1i3...|[-0.0056284335441...|
spotify:track:3EK...|[-0.4708164334297...|
spotify:track:59q...|[-0.1429600268602...|
+--------------------+--------------------+
only showing top 10 rows

We wrote the output initially to S3 because we were concerned about the MongoDB charges, and wanted to save the trained model weights because this takes long time to train.

In [ ]:
# writing the model output to s3
vectors.repartition(1).write.json('s3://spotify-recommendation/song_embedding_word2vec/')

Reading the trained vectors from S3

In [ ]:
song_embed_file = 's3://spotify-recommendation/song_embedding_word2vec/part-00000-tid-4058272997583498648-4f4c2eac-f7e1-4c42-90d8-10098e466450-44807-1-c000.json'
df_song_embed = spark.read.json(song_embed_file)
df_song_embed.cache()
df_song_embed.select('track_uri', 'vector.values').show(5)

+--------------------+--------------------+
 track_uri| values|
+--------------------+--------------------+
spotify:track:7Bw...|[0.17075945436954...|
spotify:track:3QV...|[-0.1242264881730...|
spotify:track:2wf...|[-0.1510251611471...|
spotify:track:7KX...|[-0.0403408631682...|
spotify:track:23Q...|[-0.5807483792304...|
+--------------------+--------------------+
only showing top 5 rows

In [ ]:
# Save track, vector pairs in mongodb
collection = 'song_vectors'
connection_string = f"mongodb+srv://{user_name}:{password}@{address}/{database}.{collection}"
df_song_embed.select('track_uri', 'vector.values').write.format("mongo").option("uri", connection_string).mode("append").save()

In [ ]:
# reading back from mongodb to check
collection = 'song_vectors'
connection_string = f"mongodb+srv://{user_name}:{password}@{address}/{database}.{collection}"
df_song_embed = spark.read.format('mongo').option("uri",connection_string).load()

In [ ]:
df_song_embed_mongo.count()

Out[13]: 2262292

In [ ]:
df_song_embed.cache()
df_song_embed.count()

Out[14]: 2262292

Both the counts match!

Now, the song embeddings dataframe has just track_uri and song vector. We will join with the playlist dataframe to bring in song information.

Creating a track info data frame for each track with columns, Track URI, Track Name, Album Name, Artist Name.

In [ ]:
df_track_info = df.groupBy('track_uri', 'track_name', 'album_name', 'artist_name').agg(count('pid').alias('pid_count')).sort('pid_count', ascending = False)
df_track_info.cache()
df_track_info.show(5)

+--------------------+--------------------+-------------+--------------+---------+
 track_uri| track_name| album_name| artist_name|pid_count|
+--------------------+--------------------+-------------+--------------+---------+
spotify:track:7KX...| HUMBLE.| DAMN.|Kendrick Lamar| 93148|
spotify:track:7yy...|Broccoli (feat. L...|Big Baby DRAM| DRAM| 82618|
spotify:track:3a1...| Congratulations| Stoney| Post Malone| 79974|
spotify:track:4Km...|Bad and Boujee (f...| Culture| Migos| 69998|
spotify:track:0SG...| Bounce Back| I Decided.| Big Sean| 67398|
+--------------------+--------------------+-------------+--------------+---------+
only showing top 5 rows

Joining this track information with song embeddings dataframe on track_uri.<br>
We are restricting the songs to top 100k based on number of playlist they occur in, because the tensor projector which we will use to visualize these embeddings doesn't work well with 2million song embeddings.

In [ ]:
df_joined = df_track_info.limit(100000).join(df_song_embed, 'track_uri', 'left')
df_joined.show(5)

+--------------------+--------------------+--------------------+-----------+---------+--------------------+--------------------+
 track_uri| track_name| album_name|artist_name|pid_count| _id| values|
+--------------------+--------------------+--------------------+-----------+---------+--------------------+--------------------+
spotify:track:0sE...| Kolniður| Go| Jónsi| 58|{622bc2b268528929...|[-0.1272485405206...|
spotify:track:22D...| What You Want| Live from Trona| Toro y Moi| 19|{622bc2c87d21b373...|[0.03801601380109...|
spotify:track:2Xg...|Emergency Spliff ...| Emergency Spliff| SkillinJah| 99|{622bc29368528929...|[-0.0745467990636...|
spotify:track:3ru...| Go with the Flow|Finding Nemo: The...| Marlin| 24|{622bc2cf7d21b373...|[0.23135676980018...|
spotify:track:3v8...| Can't Forget You| Ammunition| Krewella| 35|{622bc29397052242...|[-0.1594378501176...|
+--------------------+--------------------+--------------------+-----------+---------+--------------------+--------------------+
only showing top 5 rows

The next steps are: <br>
1. Exporting this file into a tsv format
2. Upload it to the tensorprojector.org
3. Visualize the embeddings